In [11]:

from ordo_primitives import *

# Assuming PRIMITIVE is a placeholder variable
#PRIMITIVE = 'PRIMITIVE'

# Use Stringify to process the JSON data


json_data_useful = {
    'main': pipelineLibrary.compute(rebalancingDate),
    'rebalancingDate': getCurrentDate(),
    'data_loader': PRIMITIVE,
    'compute_engine': PRIMITIVE,
    'orchestrator': PRIMITIVE,
    'getParams': PRIMITIVE,
    'pipelineLibrary': {
        'rebalancing': {
            'rebalancingDate': getParams(),
            'universe': data_loader.getUniverseFromTickers(["AAPL", "GOOGLE"]),
            'enrichedUniverse': data_loader.getData(universe, ["Price"], rebalancingDate.offset("-3M")),
            'targetAllocation': compute_engine.getEqualWeightAllocation(universe, rebalancingDate),
        },
        'backtest': {
            'rebalancingDate': getParams(),
            'observationDates': orchestrator.getSchedule("2020-01-01", rebalancingDate, "monthly"),
            'rebalancingResults': observationDates.map(rebalancing)
        },
        'compute': {
            'rebalancingDate': getParams(),
            'backtestResults': backtestResults(rebalancingDate),
            'performanceMetrics': compute_engine.calculatePerformance(backtestResults),
            'report': orchestrator.finalizeReport(performanceMetrics),
        }
    }
}

json_data = Stringify('json_data_useful')
print(json_data)


{'main': 'pipelineLibrary.compute(rebalancingDate)', 'rebalancingDate': 'getCurrentDate()', 'data_loader': '"PRIMITIVE"', 'compute_engine': '"PRIMITIVE"', 'orchestrator': '"PRIMITIVE"', 'getParams': '"PRIMITIVE"', 'pipelineLibrary': {'rebalancing': {'rebalancingDate': 'getParams()', 'universe': "data_loader.getUniverseFromTickers(['AAPL', 'GOOGLE'])", 'enrichedUniverse': "data_loader.getData(universe, ['Price'], rebalancingDate.offset('-3M'))", 'targetAllocation': 'compute_engine.getEqualWeightAllocation(universe, rebalancingDate)'}, 'backtest': {'rebalancingDate': 'getParams()', 'observationDates': "orchestrator.getSchedule('2020-01-01', rebalancingDate, 'monthly')", 'rebalancingResults': 'observationDates.map(rebalancing)'}, 'compute': {'rebalancingDate': 'getParams()', 'backtestResults': 'backtestResults(rebalancingDate)', 'performanceMetrics': 'compute_engine.calculatePerformance(backtestResults)', 'report': 'orchestrator.finalizeReport(performanceMetrics)'}}}


In [ ]:
from task_module import Task, TaskGraph


def create_task_graph_from_json(json_data):
    task_graph = TaskGraph()
    task_graph.parse(json_data)
    return task_graph

# Sample Usage
if __name__ == "__main__":
    task_graph = create_task_graph_from_json(json_data)
    print("Main Entry Point:", task_graph.get_entry_point())
    print("Full Task Graph:")
    print(task_graph)


Main Entry Point: Task(name='main', type='single', formula='pipelineLibrary.compute(rebalancingDate)', parents=[], siblings=['rebalancingDate', 'data_loader', 'compute_engine', 'orchestrator', 'getParams', 'pipelineLibrary'], children=[], dependencies=['rebalancingDate'], object_dependencies=['pipelineLibrary'])
Full Task Graph:
Task(name='main', type='single', formula='pipelineLibrary.compute(rebalancingDate)', parents=[], siblings=['rebalancingDate', 'data_loader', 'compute_engine', 'orchestrator', 'getParams', 'pipelineLibrary'], children=[], dependencies=['rebalancingDate'], object_dependencies=['pipelineLibrary'])
Task(name='rebalancingDate', type='single', formula='getCurrentDate()', parents=[], siblings=['main', 'data_loader', 'compute_engine', 'orchestrator', 'getParams', 'pipelineLibrary'], children=[], dependencies=[], object_dependencies=[])
Task(name='data_loader', type='single', formula='"PRIMITIVE"', parents=[], siblings=['main', 'rebalancingDate', 'compute_engine', 'orch

In [19]:
task_graph.tasks['pipelineLibrary.rebalancing.universe']

Task(name='pipelineLibrary.rebalancing.universe', type='single', formula='data_loader.getUniverseFromTickers(['AAPL', 'GOOGLE'])', parents=['pipelineLibrary', 'pipelineLibrary.rebalancing'], siblings=['pipelineLibrary.rebalancing.enrichedUniverse', 'pipelineLibrary.rebalancing.targetAllocation'], children=[], dependencies=[], object_dependencies=['data_loader'])